In [27]:
import sys
sys.path.append('../src')

import os
import requests
import json
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta
import time

from betfair import Betfair

pd.set_option('display.max_columns', None)

In [28]:
app_key = 'ABGJLOlKaLtTsMIp'

username1 = input("Email:")
password1 = input("Password:")

In [29]:
trading = betfairlightweight.APIClient(
        username1, password1, app_key
    )
trading.login_interactive()

trading.session_token

trading.keep_alive()

<KeepAliveResource>

In [30]:
# Set the start and end time to cover the entire day in UTC
today = datetime.utcnow()
start_time = today.replace(hour=0, minute=0, second=0, microsecond=0)
end_time = today.replace(hour=23, minute=59, second=59, microsecond=999999)

In [31]:
# Define filter for horse racing markets
market_filter = betfairlightweight.filters.market_filter(
    event_type_ids=['7'],
    market_countries=['GB'],
    market_type_codes=['WIN'], 
    market_start_time={'from': start_time.isoformat(),
                       'to' : end_time.isoformat()}
)

# Request market catalogue for all horse races on the date
results = trading.betting.list_market_catalogue(
    filter=market_filter,
    market_projection=["RUNNER_DESCRIPTION", 
            "RUNNER_METADATA", 
            "COMPETITION", 
            "EVENT", 
            "EVENT_TYPE", 
            "MARKET_DESCRIPTION", 
            "MARKET_START_TIME",],  # Include runner metadata
    max_results=1000  # Adjust based on expected number of races
)

In [32]:
for i in results:
        print(
            "{0} {1:d}:{2:02d} {3} ({4})".format(
                i.market_id,
                i.market_start_time.hour, 
                i.market_start_time.minute, 
                i.event.venue,  
                i.description.market_type,
            )
        )


1.232300622 13:15 Catterick (WIN)
1.232300476 13:30 Musselburgh (WIN)
1.232300628 13:45 Catterick (WIN)
1.232300483 14:00 Musselburgh (WIN)
1.232300633 14:15 Catterick (WIN)
1.232300490 14:35 Musselburgh (WIN)
1.232300639 14:50 Catterick (WIN)
1.232300503 15:05 Musselburgh (WIN)
1.232300645 15:25 Catterick (WIN)
1.232300496 15:35 Musselburgh (WIN)
1.232300572 15:40 Worcester (WIN)
1.232300651 15:55 Catterick (WIN)
1.232300511 16:05 Musselburgh (WIN)
1.232300578 16:10 Worcester (WIN)
1.232300665 16:30 Catterick (WIN)
1.232300585 16:40 Worcester (WIN)
1.232300437 17:00 Kempton (WIN)
1.232300658 17:00 Catterick (WIN)
1.232300592 17:10 Worcester (WIN)
1.232300430 17:30 Kempton (WIN)
1.232300607 17:40 Worcester (WIN)
1.232300443 18:00 Kempton (WIN)
1.232300600 18:10 Worcester (WIN)
1.232300449 18:30 Kempton (WIN)
1.232300614 18:40 Worcester (WIN)
1.232300456 19:00 Kempton (WIN)
1.232300463 19:30 Kempton (WIN)
1.232300470 20:00 Kempton (WIN)


In [33]:
import pandas as pd

# Initialize an empty list to store market and runner details
data = []

# Iterate over each market in the market catalogue
for market in results:
    market_id = market.market_id
    market_name = market.market_name
    market_start_time = market.market_start_time
    market_type = market.description.market_type

    # Extract the location/venue of the race
    event = market.event
    race_location = event.venue if event else "Unknown Location"
    
    # Iterate over each runner in the market
    for runner in market.runners:
        runner_id = runner.selection_id
        runner_name = runner.runner_name
        
        # Append a dictionary with all relevant data to the list
        data.append({
            'market_id': market_id,
            'market_start_time': market_start_time,
            'venue': race_location,
            'market_type': market_type,
            'selection_id': runner_id,
            'horse_name': runner_name,
        })

# Convert the list of dictionaries to a pandas DataFrame
df1 = pd.DataFrame(data)

# Display the DataFrame
print(df1)


       market_id   market_start_time      venue market_type  selection_id  \
0    1.232300622 2024-08-28 13:15:00  Catterick         WIN      70258001   
1    1.232300622 2024-08-28 13:15:00  Catterick         WIN      69684619   
2    1.232300622 2024-08-28 13:15:00  Catterick         WIN      68837968   
3    1.232300622 2024-08-28 13:15:00  Catterick         WIN      71273678   
4    1.232300622 2024-08-28 13:15:00  Catterick         WIN      73115482   
..           ...                 ...        ...         ...           ...   
268  1.232300470 2024-08-28 20:00:00    Kempton         WIN      62669407   
269  1.232300470 2024-08-28 20:00:00    Kempton         WIN      60237412   
270  1.232300470 2024-08-28 20:00:00    Kempton         WIN      59423612   
271  1.232300470 2024-08-28 20:00:00    Kempton         WIN      57271875   
272  1.232300470 2024-08-28 20:00:00    Kempton         WIN      68392036   

             horse_name  
0             Nala Star  
1       Stapleford Park

# I need to get the a list of id's from the market_id column and then create a loop for the list_market_book 

In [34]:
market_id_list = df1['market_id'].drop_duplicates().to_list()
len(market_id_list)

28

In [35]:
# Define a delay between requests
delay = 0.05  # Delay in seconds

# Loop through market IDs and fetch data with a delay
market_books = []
for market_id in market_id_list:
    market_book = trading.betting.list_market_book(
        market_ids=[market_id],
        price_projection=betfairlightweight.filters.price_projection(
            price_data=betfairlightweight.filters.price_data(ex_all_offers=True)
        ),
    )
    market_books.append(market_book)
    
    # Print or process the market_book as needed
    print(market_book)
    
    # Sleep to avoid overloading the API
    time.sleep(delay)

[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]


In [36]:
# Example: if market_books is a list of lists of MarketBook objects
data = []

# Iterate over each list of MarketBook objects
for book_list in market_books:
    # Iterate over each MarketBook object in the current list
    for market_book in book_list:
        market_id = market_book.market_id
        status = market_book.status
        total_matched = market_book.total_matched

        # Iterate over each runner in the market
        for runner in market_book.runners:
            runner_id = runner.selection_id
            runner_status = runner.status
            last_price_traded = runner.last_price_traded
            runner_total_matched = runner.total_matched
            
            # Append a dictionary with all relevant data to the list
            data.append({
                'market_id': market_id,
                'status': status,
                'total_matched': total_matched,
                'selection_id': runner_id,
                'status': runner_status,
                'morning_price': last_price_traded,
            })



# Convert the list of dictionaries into a DataFrame
df2 = pd.DataFrame(data)

df2 = df2[['market_id', 'selection_id', 'status', 'morning_price', 'total_matched']]
df2

,market_id,selection_id,status,morning_price,total_matched
0,1.232300622,70258001,ACTIVE,3.05,15019.49
1,1.232300622,69684619,ACTIVE,3.65,15019.49
2,1.232300622,68837968,ACTIVE,5.20,15019.49
3,1.232300622,71273678,ACTIVE,12.50,15019.49
4,1.232300622,73115482,ACTIVE,21.00,15019.49
...,...,...,...,...,...
268,1.232300470,62669407,ACTIVE,19.00,5754.56
269,1.232300470,60237412,ACTIVE,25.00,5754.56
270,1.232300470,59423612,ACTIVE,44.00,5754.56
271,1.232300470,57271875,REMOVED,NaN,5754.56


# join df1 and df2

In [37]:
df3 = df1.merge(df2, on = ['market_id', 'selection_id'], how = 'left')

df3['market_id'] = df3['market_id'].astype(str)


current_date = datetime.now().strftime('%Y-%m-%d')
file_name = f"{current_date}_data.csv"


df3.to_csv('../data/daily/' + file_name, index=False)
df3

,market_id,market_start_time,venue,market_type,selection_id,horse_name,status,morning_price,total_matched
0,1.232300622,2024-08-28 13:15:00,Catterick,WIN,70258001,Nala Star,ACTIVE,3.05,15019.49
1,1.232300622,2024-08-28 13:15:00,Catterick,WIN,69684619,Stapleford Park,ACTIVE,3.65,15019.49
2,1.232300622,2024-08-28 13:15:00,Catterick,WIN,68837968,Beaujolais Nouveau,ACTIVE,5.20,15019.49
3,1.232300622,2024-08-28 13:15:00,Catterick,WIN,71273678,Montana Girl,ACTIVE,12.50,15019.49
4,1.232300622,2024-08-28 13:15:00,Catterick,WIN,73115482,Youvegotoshowmeluv,ACTIVE,21.00,15019.49
...,...,...,...,...,...,...,...,...,...
268,1.232300470,2024-08-28 20:00:00,Kempton,WIN,62669407,Turing,ACTIVE,19.00,5754.56
269,1.232300470,2024-08-28 20:00:00,Kempton,WIN,60237412,Burlington House,ACTIVE,25.00,5754.56
270,1.232300470,2024-08-28 20:00:00,Kempton,WIN,59423612,Saratoga Star,ACTIVE,44.00,5754.56
271,1.232300470,2024-08-28 20:00:00,Kempton,WIN,57271875,Cloud Free,REMOVED,NaN,5754.56
